# Tutorial

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os, sys, pathlib
ROOT = pathlib.Path(os.getcwd()).parent.parent
sys.path.append(str(ROOT))

from utils import helpers, dataloading, prompting

### Load Model

In [21]:
model, tokenizer = helpers.load_model(model_name="gpt2-small", device="cpu")

loaded gpt2-small on cpu


### Load Data

In [18]:
class SynthFacts(dataloading.Data_Loader):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def load(self, prompt_type):
        ents_ranks = [{"ents": ["horrible", "bad", "okay", "good", "great", "awesome"], "ranks": [0, 1, 2, 3, 4, 5]},
                      {"ents": ["1", "10", "100", "500", "1000", "10000"], "ranks": [0, 1, 2, 3, 4, 5]}]

        data = {"pair": [{"prompt": 'Does the adjective "{a}" convey a more positive sentiment than "{b}"? {answ}',
                        "tok": {"Yes": "ĠYes", "No": "ĠNo"}},
                        {"prompt": 'Is "{a}" a larger number than "{b}"? {answ}', "tok": {"Yes": "ĠYes", "No": "ĠNo"}}],
                "point": [{"prompt": 'The sentiment of the adjective "{x}" is {answ}', "tok": {"[MASK]": "[MASK]"}},
                            {"prompt": "The cardinality of the number {x} is {answ}", "tok": {"[MASK]": "[MASK]"}}],
                "point_scale": [{"prompt": 'On a scale from 0 to 10, the sentiment of the adjective "{x}" is {answ}', "tok": {"[MASK]": "[MASK]"}},
                                  {"prompt": "On a scale from 0 to 10, the cardinality of the number {x} is {answ}", "tok": {"[MASK]": "[MASK]"}}],
                "list": [{"prompt": 'Order the following adjectives by sentiment. Adjectives: {x}. The correct ordering is:', "tok": {'{alpha}'}}, ## alpha means: (alpha) option A, B ...
                        {"prompt": 'Order the following numbers by cardinality. Numbers: {x}. The correct ordering is:', "tok": {'{alpha}'}}] ## just list the numbers....
                }
        data = [{**prompt_tok, **ents_ranks[i]} for i, prompt_tok in enumerate(data[prompt_type])]
        return data


class SynthContext(dataloading.Data_Loader):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def load(self, prompt_type):

        ents_ranks = [{"ents": ["brown", "green", "purple", "yellow", "red", "blue"], "ranks": [0, 1, 2, 3, 4, 5]},
                      {"ents": ["Emily", "Muhammad", "An", "Tom", "Sam", "Jenny"], "ranks": [0, 1, 2, 3, 4, 5]}]

        contexts = ["Most students selected blue as their favourite color, followed by red, then yellow. Brown ranked lowest, green second lowest and purple third lowest. ",
                   "An owns 100 dollar, Tom owns 50 dollars more and Sam 75 dollars more. Jenny is the richest owning 1000 dollar. Emily and Muhammad are at the lower end owning only 5 dollar and 10 dollars respectively. "]

        data = {"pair": [{"prompt": contexts[0] + "Is the color {a} more popular than {b}? {answ}", "tok": {"Yes": "ĠYes", "No": "ĠNo"}},
                    {"prompt": contexts[1] + 'Is {a} richer than {b}? {answ}',"tok": {"Yes": "ĠYes", "No": "ĠNo"}}],
                "point": [{"prompt": contexts[0] + "The popularity of the color {x} is {answ}", "tok": {"[MASK]": "[MASK]"}},
                            {"prompt": contexts[1] + 'The wealth of {x} is {answ}',"tok": {"[MASK]": "[MASK]"}}],
                "point_scale": [{"prompt": contexts[0] + "On a scale from 0 to 10, the popularity of the color {x} is {answ}","tok": {"[MASK]": "[MASK]"}},
                             {"prompt": contexts[1] + 'On a scale from 0 to 10, the wealth of {x} is {answ}', "tok": {"[MASK]": "[MASK]"}}],
                "list": [{"prompt": contexts[0] + 'Order the colors {x} by popularity. The correct ordering is:', "tok": {'{alpha}'}}, ## alpha means: (alpha) option A, B ...
                        {"prompt": contexts[1] + 'Order the persons {x} by wealth. The correct ordering is:', "tok": {'{alpha}'}}]## just list the numbers....
                }

        data = [{**prompt_tok, **ents_ranks[i]} for i, prompt_tok in enumerate(data[prompt_type])]
        return data

In [22]:
prompt_type = "pair"  #@param ["pair", "point", "point_scale", "list"]
dataset = "SynthFacts" #@param ["SynthFacts", "SynthContext"]
add_exemplar = False #@param {type:"boolean"}
option_marker = "\"" #@param {type:"string"}

if dataset == "SynthFacts":
  data = SynthFacts(prompt_type=prompt_type, option_marker='"', add_examplar=add_exemplar).data
elif dataset == "SynthContext":
  data = SynthContext(prompt_type=prompt_type, option_marker='"', add_examplar=add_exemplar).data

prompter = prompting.Prompter(model, tokenizer, print_prompt=True)

print("\n")
for task in data:
  prompt, ents = task["prompt"], task["ents"]
  print(f"items: {ents}, prompt: {prompt}")

prompt_type: point_scale, kwargs: {'option_marker': '"', 'add_examplar': False}
GPT2LMHeadModel
items: ['horrible', 'bad', 'okay', 'good', 'great', 'awesome'], prompt: On a scale from 0 to 10, the sentiment of the adjective "{x}" is {answ}
items: ['1', '10', '100', '500', '1000', '10000'], prompt: On a scale from 0 to 10, the cardinality of the number {x} is {answ}


### Execute prompting

In [20]:
helpers.evaluate(prompter, data, return_type=[""], tie=False)


[('On a scale from 0 to 10, the sentiment of the adjective "horrible" is [MASK]', 0), ('On a scale from 0 to 10, the sentiment of the adjective "bad" is [MASK]', 5), ('On a scale from 0 to 10, the sentiment of the adjective "okay" is [MASK]', 2), ('On a scale from 0 to 10, the sentiment of the adjective "good" is [MASK]', 4), ('On a scale from 0 to 10, the sentiment of the adjective "great" is [MASK]', 1), ('On a scale from 0 to 10, the sentiment of the adjective "awesome" is [MASK]', 3)]

ranks: {'kendall': 0.0667}
pairwise: {'accuracy': 0.5333}
predicted ranks: [0, 5, 2, 4, 1, 3] true ranks: [0, 1, 2, 3, 4, 5]

[('On a scale from 0 to 10, the cardinality of the number 1 is [MASK]', 2), ('On a scale from 0 to 10, the cardinality of the number 10 is [MASK]', 4), ('On a scale from 0 to 10, the cardinality of the number 100 is [MASK]', 5), ('On a scale from 0 to 10, the cardinality of the number 500 is [MASK]', 0), ('On a scale from 0 to 10, the cardinality of the number 1000 is [MASK]'

In [16]:
model.device

device(type='cpu')